In [3]:
# Install Tableau Server Client (TSC)
!pip install tableauserverclient requests

# **Login to Tableau Cloud Site**

In [33]:
import tableauserverclient as TSC
import requests
from google.colab import userdata


my_server_url = 'https://10ax.online.tableau.com/'
my_token_name = userdata.get('TC_token_name')
my_token_secret = userdata.get('TC_token_secret')
my_site_id = userdata.get('TC_site_id')
my_data_source = "cpat_results"


tableau_auth = TSC.PersonalAccessTokenAuth( token_name=my_token_name
                                           ,personal_access_token=my_token_secret
                                           ,site_id=my_site_id)
server = TSC.Server(my_server_url, use_server_version=True)

# Sign in to Tableau Cloud

server.auth.sign_in_with_personal_access_token(tableau_auth)
print('[Logged in successfully to {}]'.format(my_server_url))

[Logged in successfully to https://10ax.online.tableau.com/]


# **Retrieve only the requested Datasource and Populate Connection Details**

In [39]:
import pandas as pd
from pandas import json_normalize

#Query the Metadata API and store the response in resp
#Add filteration clause to consider only the requested connection

#Set the query https://help.tableau.com/current/api/metadata_api/en-us/docs/meta_api_examples.html
query = """
{
  databaseServersConnection(filter: {hostName: "mysql-rfam-public.ebi.ac.uk"}) {
    nodes {
      luid
      name
      hostName
      port
      downstreamWorkbooks {
        luid
        name
        projectName
        projectLuid
      }
      downstreamDatasources {
        luid
        name
        projectName
      }
    }
  }
}
    """


resp = server.metadata.query(query)

***I have created 1 datasource using this connection and two additional workbooks. One workbook has a connection to the published datasource created earlier. The second workbook has an embedded connection.***

***This's to evaluate the effort needed for fixing the 3 objects.***

In [43]:
from tabulate import tabulate


# Extract & flatten
connections = resp["data"]["databaseServersConnection"]["nodes"]
flat_workbooks = []
flat_datasources = []

for connection in connections:
    connection_info = {
        "connection_id": connection["luid"],
        "connection_name": connection["name"],
        "host_name": connection["hostName"],
        "port": connection["port"]
    }

    # Direct downstream workbooks (from connection)
    for wb in connection.get("downstreamWorkbooks", []):
        flat_workbooks.append({
            **connection_info,
            "workbook_id": wb["luid"],
            "workbook_name": wb["name"],
            "project_name": wb["projectName"],
            "project_luid": wb.get("projectLuid")
        })

    # Datasources
    for ds in connection.get("downstreamDatasources", []):
        ds_info = {
            **connection_info,
            "datasource_id": ds["luid"],
            "datasource_name": ds["name"],
            "datasource_project_name": ds["projectName"]
        }
        flat_datasources.append(ds_info)


# Convert to DataFrames
df_connection_workbooks = pd.DataFrame(flat_workbooks)
df_datasources = pd.DataFrame(flat_datasources)

# Show sample output
print("🔗 Connection ➝ Workbooks:")
print(tabulate(df_connection_workbooks, headers='keys', tablefmt='psql'))
print("\n")

print("🗄️ Connection ➝ Datasources:")
print(tabulate(df_datasources, headers='keys', tablefmt='psql'))
print("\n")



🔗 Connection ➝ Workbooks:
+----+--------------------------------------+-------------------+-----------------------------+--------+--------------------------------------+-----------------+----------------+--------------------------------------+
|    | connection_id                        | connection_name   | host_name                   |   port | workbook_id                          | workbook_name   |   project_name | project_luid                         |
|----+--------------------------------------+-------------------+-----------------------------+--------+--------------------------------------+-----------------+----------------+--------------------------------------|
|  0 | 734528f6-cca9-4dcd-8077-a255139f575d | Rfam              | mysql-rfam-public.ebi.ac.uk |   4497 | 20137fcc-cbbe-4d46-8fb4-14a12de618a3 | Genome          |         202502 | 85b628ee-8006-4e19-97e3-86f379a901d8 |
|  1 | 734528f6-cca9-4dcd-8077-a255139f575d | Rfam              | mysql-rfam-public.ebi.ac.uk |   4497

***Let's retreive connection details for all 3 objects i.e Username "Value need to be fixed". The 3 objects have the same connection details***

In [57]:
#Loop over retreived datasources
for ds_id in df_datasources['datasource_id'].unique():
  ds = server.datasources.get_by_id(ds_id)
  print(f"DS Name:\t{ds.name}")
  server.datasources.populate_connections(ds)
  connection = ds.connections[0]
  print(f"Connection DS Name:\t{connection.datasource_name}")
  print(f"Username:\t{connection.username}")
  print("\n")

#Loop over retreived workbooks
for wb_id in df_connection_workbooks['workbook_id'].unique():
  wb = server.workbooks.get_by_id(wb_id)
  print(f"WB Name:\t{wb.name}")
  server.workbooks.populate_connections(wb)
  connection = wb.connections[0]
  print(f"Connection DS Name:\t{connection.datasource_name}")
  print(f"Username:\t{connection.username}")
  print("\n")

DS Name:	genome
Connection DS Name:	genome
Username:	rfamros


WB Name:	Genome
Connection DS Name:	genome
Username:	


WB Name:	Genemoe_2
Connection DS Name:	genome (local copy)
Username:	rfamrox




**The 1st Workbook i.e `Genome` don't have a value for the username as It has a connection to a published datasource i.e `genome`**

**The 2nd Workbbok is the one has an embedded connection i.e `genome (local copy)`**

# **Update Connection Details**

In [62]:
#Loop over retreived datasources to fix username
for ds_id in df_datasources['datasource_id'].unique():
  ds = server.datasources.get_by_id(ds_id)
  print(f"DS Name:\t{ds.name}")
  server.datasources.populate_connections(ds)
  connection = ds.connections[0]
  # Assign the correct value for Username
  connection.username = 'rfamro'
  # update datasource connection
  server.datasources.update_connection(ds, connection)
  print(f"Connection Username for {connection.datasource_name} is Fixed")
  print("\n")

#Loop over retreived workbooks to fix username
for wb_id in df_connection_workbooks['workbook_id'].unique():
  wb = server.workbooks.get_by_id(wb_id)
  print(f"WB Name:\t{wb.name}")
  server.workbooks.populate_connections(wb)
  connection = wb.connections[0]
  if connection.username:
    # Assign the correct value for Username
    connection.username = 'rfamro'
    # update datasource connection
    server.workbooks.update_connection(wb, connection)
    print(f"Connection Username for {connection.datasource_name} is Fixed")
  print("\n")

DS Name:	genome
Connection Username for genome is Fixed


WB Name:	Genome


WB Name:	Genemoe_2
Connection Username for genome (local copy) is Fixed




# **Let's Validate the updated connection**

In [63]:
#Loop over retreived datasources
for ds_id in df_datasources['datasource_id'].unique():
  ds = server.datasources.get_by_id(ds_id)
  print(f"DS Name:\t{ds.name}")
  server.datasources.populate_connections(ds)
  connection = ds.connections[0]
  print(f"Connection DS Name:\t{connection.datasource_name}")
  print(f"Username:\t{connection.username}")
  print("\n")

#Loop over retreived workbooks
for wb_id in df_connection_workbooks['workbook_id'].unique():
  wb = server.workbooks.get_by_id(wb_id)
  print(f"WB Name:\t{wb.name}")
  server.workbooks.populate_connections(wb)
  connection = wb.connections[0]
  print(f"Connection DS Name:\t{connection.datasource_name}")
  print(f"Username:\t{connection.username}")
  print("\n")

DS Name:	genome
Connection DS Name:	genome
Username:	rfamro


WB Name:	Genome
Connection DS Name:	genome
Username:	


WB Name:	Genemoe_2
Connection DS Name:	genome (local copy)
Username:	rfamro




# **Sign out from Tableau Cloud**

In [ ]:
# Sign Out from Tableau Cloud

server.auth.sign_out()

# **Refrences:**

1. https://tableau.github.io/server-client-python/docs/api-ref#data-sources
2. https://tableau.github.io/server-client-python/docs/api-ref#connections

